# ClipCard Agent Harness — Industrial / Aviation


# ClipCard Notebook Plan (auto-generated 2025-10-13)

This notebook is a scaffold for the **ClipCard (Risk & Recheck)** seedpack.

- **Base schema** (`schema/clipcard.schema.v1.json`) is required.
- **Extended schema** (`schema/clipcard.schema.v1e.json`) is optional.
- Works with CSVs under `data/clipmap/*.csv` and any `**/*.clipcard.json` in your repo.

## Inputs
- `schema/clipcard.schema.v1.json` (required)
- `schema/clipcard.schema.v1e.json` (optional)
- `data/clipmap/cards.csv` and `data/clipmap/events.csv` (optional; created by CI)
- Any `**/*.clipcard.json` examples you add

## Outputs
- Console metrics and/or CSVs in `notebooks/_out/`
- Plots (if matplotlib installed)
- Notes you can copy into Field Reports

> Optional installs (inside a notebook cell):
>
> ```
> %pip install pandas jsonschema matplotlib
> ```



## Rule-based agent harness (no LLMs)

- Validates cards against **base** (required) and **extended** (optional) schemas
- Checks guardrails: hazard specificity, steward present, measurable kill criteria
- You can add domain-specific guardrails later


In [ ]:

# %pip install jsonschema
import os, glob, json

try:
    from jsonschema import Draft7Validator
except Exception as e:
    Draft7Validator = None
    print("jsonschema not available; schema checks will be skipped.")

def load_json(p):
    with open(p,"r",encoding="utf-8") as f: return json.load(f)

base_schema_path = os.path.join("schema","clipcard.schema.v1.json")
ext_schema_path  = os.path.join("schema","clipcard.schema.v1e.json")

base = load_json(base_schema_path) if os.path.exists(base_schema_path) else None
ext  = load_json(ext_schema_path)  if os.path.exists(ext_schema_path)  else None

cards = glob.glob("**/*.clipcard.json", recursive=True)
print(f"Found {len(cards)} ClipCard(s).")

fail, warn = 0, 0
for f in cards:
    d = load_json(f)
    if Draft7Validator and base:
        v = Draft7Validator(base)
        errs = sorted(v.iter_errors(d), key=lambda e: e.path)
        if errs:
            print(f"[ERROR] {f}: " + "; ".join([e.message for e in errs])); fail += 1
    if Draft7Validator and ext:
        ve = Draft7Validator(ext)
        errs = sorted(ve.iter_errors(d), key=lambda e: e.path)
        if errs:
            print(f"[WARN]  {f} (v1e): " + "; ".join([e.message for e in errs])); warn += 1
    # Guardrails
    hazard_ok = bool(d.get("hazard","").strip())
    steward_ok = bool((d.get("steward") or {}).get("primary"))
    kc = d.get("kill_criteria") or []
    kc_ok = any(isinstance(x, dict) and x.get("condition") and x.get("action") for x in kc)
    if not (hazard_ok and steward_ok and kc_ok):
        print(f"[WARN]  {f}: guardrails (hazard={hazard_ok}, steward={steward_ok}, measurable_kill={kc_ok})")
        warn += 1

print(f"Done. base_fail={fail}, warn={warn}")
